# Calculate the Leontief model

### Objectives
- Understand the format of input-output tables
- Interpret the Leontief inverse
- Apply demand and price driven models
- Calculate the carbon footprint

## Exercise

A fictitious nation has a very basic national economy consisting of five sectors, each producing a single type of goods for sale: rice, beef, electricity, car, and insurance. The following data were obtained from the nation's Bureau of Statistics:
Inter-industry transactions and value added (unit: €/year):

In [1]:
# import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
plt.style.use('bmh')

# # Import data

# CBS_IO = pd.ExcelFile(f"data/Input-Output tables 2015-2019.xlsx", engine="openpyxl")

#### Inter-industry transactions and value added (unit: €/year):

In [3]:
# Products are the rows, industry are the columns
Z = np.array([
    [100, 1900, 0, 0, 0],
    [0, 1000, 0, 0, 0],
    [1600, 2500, 900, 1000, 4000],
    [500, 1500, 1500, 1000, 1500],
    [300, 700, 3000, 2000, 1000]
], dtype=np.int32)
Z

Z = pd.DataFrame(
     Z,
     index=pd.Index(["rice", "beef", "electricity", "car", "insurance"], name="product"),
     columns=pd.Index(["rice farm", "cattle farm", "power plant", "car maker", "insurance company"], name="industry")
 )
Z

industry,rice farm,cattle farm,power plant,car maker,insurance company
product,,,,,
rice,100,1900,0,0,0
beef,0,1000,0,0,0
electricity,1600,2500,900,1000,4000
car,500,1500,1500,1000,1500
insurance,300,700,3000,2000,1000


#### The value of finished products sold (final consumption,€/year):

In [4]:
# Value added (last row of the first table)
v = np.array([2500, 2400, 9600, 16000, 8500], dtype=np.int32)
v = pd.DataFrame(v, columns=["Value added"], index=pd.Index(["rice farm", "cattle farm", "power plant", "car maker", "insurance company"], name="industry")).T
v

industry,rice farm,cattle farm,power plant,car maker,insurance company
Value added,2500,2400,9600,16000,8500


#### Final value of the products sold

In [5]:
y = np.array([3000, 9000, 5000, 14000, 8000], dtype=np.int32)
y = pd.DataFrame(y,  index=pd.Index(["rice", "beef", "electricity", "car", "insurance"], name="product"), columns=["Final Demand"])
y

,Final Demand
product,
rice,3000
beef,9000
electricity,5000
car,14000
insurance,8000


#### The direct water use and CO2 emissions by sector. (Note: this is the environmental extension matrix F)

In [6]:
F = np.array([
    [10000, 2000, 15000, 1000, 750],
    [50, 500, 7500, 300, 15]
], dtype=np.int32)

F = pd.DataFrame(F, 
                 index=pd.Index(["Water Use (L)", "CO2 Emissions (kg)"], name="extensions"), 
                 columns=pd.Index(["rice farm", "cattle farm", "power plant", "car maker", "insurance company"], name="industry"))
F


industry,rice farm,cattle farm,power plant,car maker,insurance company
extensions,,,,,
Water Use (L),10000,2000,15000,1000,750
CO2 Emissions (kg),50,500,7500,300,15


#### Total outputs: intermediate output + final product output

Multiply the products/industry table (Z) by shaped array of ones...
i = np.ones((5,1))  # shape (5, 1)
z_sum = np.dot(Z, i)  # shape (5, 1)

... Or just get the sum of the rows while keeping the dimensions.
Z_sum = Z.sum(axis=1)  # Wrong dimensions: shape (5, )

In [27]:
Z_sum = Z.sum(axis=1)  # shape (5, 1)
x_out = Z_sum + y.T
x_out.index = ["Product output"]
x_out

product,rice,beef,electricity,car,insurance
Product output,5000,10000,15000,20000,15000


In [28]:
# Total inputs: intermediate input + value added
Z_sum = Z.sum(axis=0)  # shape (1, 5)
x_in = Z_sum + v
x_in.index = ["Product input"]
x_in

industry,rice farm,cattle farm,power plant,car maker,insurance company
Product input,5000,10000,15000,20000,15000


In [106]:
# Value added divided by total output
f_va = v * 1/x_out.values
f_va

industry,rice farm,cattle farm,power plant,car maker,insurance company
Value added,0.5,0.24,0.64,0.8,0.566667


In [109]:
# Calculate environmental intensities
f_all = F * 1/x_out.values

# add/concatenate the value added intensity
f_all = pd.concat([pd.DataFrame(f_all), f_va], axis=0)
f_all

industry,rice farm,cattle farm,power plant,car maker,insurance company
Water Use (L),2.00,0.20,1.00,0.050,0.050000
CO2 Emissions (kg),0.01,0.05,0.50,0.015,0.001000
Value added,0.50,0.24,0.64,0.800,0.566667


In [112]:
# Calculate the leontief inverse matrix

# Create the technical coefficient matrix A first
A = Z / x_out.transpose()
# Create an identity matrix the same shape as A
I = np.identity(5)
# Use numpy linear algebra inverse function to get L = (I - A)^-1
L = np.linalg.inv(I - A)
L

ValueError: Unable to coerce to DataFrame, shape must be (6, 10): given (5, 5)

In [ ]:
# Task 4 Insurance's water multiplier
f_tot_water = f_water.dot(L)
f_tot_water

: 

In [ ]:
# We can also store the information we have about all of the processes
f_tot_water_p = np.diagflat(f_water).dot(L)
f_tot_water_p

: 

In [ ]:
# And when we sum the columns, we find that we get the same result.
np.isclose(f_tot_water_p.sum(axis=0), f_tot_water)

: 

In [ ]:
# Compare f_total_water (direct + indirect) vs. f_water (direct)
print(f"Total water (direct + indirect):\n{f_tot_water}\n\nDirect water only:\n{f_water}")

: 

In [ ]:
# Carbon footprint: We use the calculated co2 and perform a dot multiplication:
# First with the leontief inverse matrix and then with the value of the products sold.
EF_co2 = f_co2.dot(L).dot(y)
EF_co2

: 

In [ ]:
# We can also just combine all of the footprints and calculate the same.
EF = f_all.dot(L).dot(y)
EF  # In order: Water, CO2 and value added by final demand

: 

In [ ]:
# Carbon footprint related to final demand:
EF_co2_exp = f_co2 @ L @ np.diagflat(y)
EF_co2_exp

: 

In [ ]:
# Carbon footprint traced to producing sector
EF_co2_p = np.diagflat(f_co2) @ L @ y
EF_co2_p

: 

In [ ]:
# Combining the two calculation above, we have all of the information in one place
EF_co2_exp_p = np.diagflat(f_co2) @ L @ np.diagflat(y)
EF_co2_exp_p

: 

In [ ]:
# Lets check if everything comes up correctly:
assert EF_co2 == EF_co2_p.sum()
assert EF_co2 == EF_co2_exp.sum()
assert EF_co2 == EF_co2_exp_p.sum()
# If no errors show, all of the matrixes sum to the same value as EF_co2

: 

In [ ]:
# Lets also check that the EF_co2_exp_p can sum to the same values as its parts
# In this case we use 'isclose' because comparing floats is difficult to do exactly.
assert np.isclose(EF_co2_p, EF_co2_exp_p.sum(axis=1, keepdims=True)).all()
assert np.isclose(EF_co2_exp, EF_co2_exp_p.sum(axis=0, keepdims=True)).all()
# If no errors show, the EF_co2_exp_p matrix sums to the same/similar values as EF_co2_p and EF_co2_exp.

: 

In [ ]:
# Comparing float values doesn´t always work.
EF_co2_exp_p.sum(axis=1, keepdims=True) == EF_co2_p

: 

: 